In order for the researcher to generate even better queries it needs to know which search functions are avaialble to it. 
- Using the Prompty **tools** parameter an LLM can choose from functions described in a json file. 
- We can add information about which functions (sometimes called tools), the LLM has access to in a **functions.json** file. 
- Information from a json file is passed to prompty using the  *${file:functions.json}* format. 

In the case of the researcher we have a **functions.json** file with descriptions of 3 functions:
- find_information
- find_entities
- find_news

 [functions.json](./researcher/functions.json)




---

> [researcher-2.prompty](researcher/researcher-2.prompty)



In [1]:
import prompty
import prompty.azure
import os

instructions = "Can you find the best educational material for learning Python programming?"
prompty.execute(os.getcwd() + "/researcher/researcher-2.prompty", inputs={"instructions": instructions})

[ToolCall(id='call_mYW3sRKNQ9HSxDt67VIxzFIF', name='find_information', arguments='{"query":"best educational material for learning Python programming","market":"en-US"}')]


>   - In [researcher-2.prompty](researcher/researcher-2.prompty) *${file:functions.json}* has been added to **tools** under the *parameters* section in the  file.

In the result from running the Prompty file we saw that the **find_information** function was selected. 
- The LLM used the **instruction** we gave it and the **descriptions** it saw in *functions.json* to pick which function to call.  
- It also figured out which parameter values should be passed to the function. 

We can influence which function is called by being more specific about the instructions we give the LLM.



---

In [2]:
instructions = "Who is the person who invented the Python programming language?"
prompty.execute(os.getcwd() + "/researcher/researcher-2.prompty", inputs={"instructions": instructions})

[ToolCall(id='call_qeiRb5pcV69xezmJWFVHrWRD', name='find_entities', arguments='{"query":"inventor of Python programming language","market":"en-US"}')]

> **find_entities** function was selected by the LLM based on:
> 1. **The description of the function** in [functions.json](./researcher/functions.json) 
> 2. The **instructions** we passed to it. 

<br>

In [4]:
instructions = "Find the latest news about Telus?"
prompty.execute(os.getcwd() + "/researcher/researcher-2.prompty", inputs={"instructions": instructions})

[ToolCall(id='call_xiy5fyI0SHvMidri5ADUTQT0', name='find_news', arguments='{"query":"Telus","market":"en-US"}')]

**find_news function description:**
<details>

```json
{
    "type": "function",
    "function": {
      "name": "find_news",
      "description": "Finds news on the web given a query. This function uses the Bing Search API to find news on the web given a query. The response includes the most relevant news articles from the web and should be used if you're looking for news.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "An optimal search query to find news on the web using the Bing Search API"
          },
          "market": {
            "type": "string",
            "description": "The market to search in, e.g. en-US - it should match the language of the query"
          }
        },
        "required": [
          "query"
        ]
      }
    }
  }
  ```
  <details>



When we execute a Prompty file that has a **functions.json** file added to the **tools** parameter, the LLM returns a **list of Tool Calls** (also known as function calls) that look like this:

```python
[ToolCall(id='call_JtomZ3gCGHEa5MBxy6M3vypv', name='find_entities', arguments='{"query":"inventor of Python programming language","market":"en-US"}')]
```

- The Python code for the functions described in *functions.json* can be found in the [researcher3.py](researcher/researcher3.py) file. 
- These functions will pass the query and market code to the Bing Search API.

To put everything together the **research** function in [researcher3.py](researcher/researcher3.py) calls:
-  an **execute_researcher_prompty** function that has the code we saw earlier to execute the prompty file 
- an **execute_function_calls** function that runs code to execute all the functions calls

In [5]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/researcher'))

from researcher3 import execute_researcher_prompty, execute_function_calls

instructions = "Can you find the best educational material for learning python programming?"

# Execute the researcher prompty to get a list of functions calls
function_calls = execute_researcher_prompty(instructions=instructions)

# Execute the function calls
research = execute_function_calls(function_calls)
research

[{'id': 'call_8g3sVeVc5eBJ89LTNld68P8w',
  'function': 'find_information',
  'arguments': {'query': 'best educational material for learning python programming',
   'market': 'en-US'},
  'result': {'pages': [{'url': 'https://hackr.io/blog/best-python-books-for-beginners-and-advanced-programmers',
     'name': 'The 15 Best Python Books for Beginners & Advanced Coders [2024] - Hackr',
     'description': 'In this article, we share the 15 best Python books in 2024. Whether you’re new to Python or an experienced Pythonista looking to boost your skills, we’ve included Python books for beginners and pros.. As the number 3 language among developers, python books can be the ideal way to learn Python for data science, web development, and more.. And sure, you can find answers to most coding ...'},
    {'url': 'https://www.geeksforgeeks.org/best-python-books/',
     'name': '9 Best Python Books For Beginners and Experts [2025]',
     'description': 'Some programmers say that Python programming la